In [83]:
import pandas as pd
import re
import pickle


from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import stopwords
from autocorrect import Speller
import sklearn
import numpy as np
import matplotlib as plt
import pycaret
import nltk
import spacy

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.


In [84]:
#Functions used in this script

# created but can assume these companies spelled it right in there TOA
def spelling(text):
    
    spell = Speller(lang='en')
    return str(spell(text))
    
def tokening(text):
    
    #nltk.download('punkt')
    tokenized_word=word_tokenize(text)
    return tokenized_word
    
def removing_stopwords(text):
    
    #nltk.download('punkt')
    tokenized_word1=tokening(text)
    #nltk.download('stopwords')
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in tokenized_word1:
        if w not in stop_words:
            filtered_sent.append(w)
    return " ".join(filtered_sent)



def Lemmating(text):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    tokenized_word1=tokening(text)
    stemmed_words=[]
    for w in tokenized_word1:
        stemmed_words.append(lemmatizer.lemmatize(w))
    return " ".join(stemmed_words)

def pos_tag(text):
    #nltk.download('averaged_perceptron_tagger')
    tokenized_word1=tokening(text)
    pos=nltk.pos_tag(removing_stopwords(text1))
    return pos
    
# function called for cleaning html tags from text
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext


In [85]:
# import data
data = pd.read_csv('DATA\datapoints\datapoint_3.csv')

In [86]:
data = data[data['QuoteText'].notna()]
data = data[data['Index Topic'].notna()]

In [87]:
data.describe()

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
count,909,909,909,909,909,909
unique,256,4,904,105,43,17
top,brainly,bad,"The company reserves the right to change, reda...",Privacy Policy,There is a date of the last update of the terms,Cookies
freq,14,475,2,303,122,145


In [88]:
data.head()

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
0,scratch,neutral,<p>\n<i>The Scratch Privacy Policy was last up...,Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms
1,trello,bad,\n</ul>Sharing with third parties <p>We share ...,Privacy Policy,Third parties may be involved in operating the...,Third Parties
2,trello,bad,"Send ""Do Not Track"" Signals:</b> Some browsers...",Privacy Policy,This service ignores the Do Not Track (DNT) he...,Cookies
4,trello,neutral,These Cloud Terms of Service will go into effe...,Atlassian Cloud Terms of Service,There is a date of the last update of the terms,Notice of Changing Terms
6,trello,bad,5.4.</strong>\n<u>Your Indemnity</u>. You will...,Atlassian Cloud Terms of Service,"You agree to defend, indemnify, and hold the s...",Governance


In [89]:
## Makes a new data frame of data by splitting any phrases into sentences and lining it up with the same label
sentences = []
for row in data.itertuples():
    for sentence in sent_tokenize(row[3]):
        if type(sentence) != float:
            if sentence != '':
                sentences.append((row[1], row[6], row[2], sentence))
data_sentenced = pd.DataFrame(sentences, columns=['Company', 'Label', 'Punitive', 'Sentence'])

In [90]:
#remove all tags like <p> etc. in the data frame "data_sentenced"

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: cleanhtml(x['Sentence']), axis=1)

In [91]:
# remove any non alpha numeric

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: re.sub('[^0-9a-zA-Z]+', ' ', x['Sentence']), axis=1)

In [92]:
#remove stop words

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: removing_stopwords(x['Sentence']), axis=1)

In [93]:
#Lemmaing words

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: Lemmating(x['Sentence']), axis=1)

In [94]:
data_sentenced.describe()

,Company,Label,Punitive,Sentence
count,1581,1581,1581,1581
unique,256,17,4,1551
top,google,Cookies,bad,
freq,43,268,917,11


In [95]:
data_sentenced.describe(include = 'O')

,Company,Label,Punitive,Sentence
count,1581,1581,1581,1581
unique,256,17,4,1551
top,google,Cookies,bad,
freq,43,268,917,11


In [96]:
data_sentenced.head()

,Company,Label,Punitive,Sentence
0,scratch,Notice of Changing Terms,neutral,The Scratch Privacy Policy last updated May 23...
1,trello,Third Parties,bad,Sharing third party We share information third...
2,trello,Third Parties,bad,If service provider need access information pe...
3,trello,Third Parties,bad,We may also share information third party agre...
4,trello,Third Parties,bad,Doing may give third party apps access account...


In [97]:
# change label to binary either bad or not bad
data_sentenced['Punitive'].replace(['blocker', 'neutral', 'good'], 0, inplace=True)
data_sentenced['Punitive'].replace(['bad'], 1, inplace=True)

In [98]:
#place into series for input and label

X_lab, y_lab = data_sentenced['Sentence'], data_sentenced['Label']

X_lab_train, X_lab_val, y_lab_train, y_lab_val = sklearn.model_selection.train_test_split(X_lab, y_lab, test_size=0.2, random_state=42)

In [99]:
#place into series for input and label

X_pun, y_pun = data_sentenced['Sentence'], data_sentenced['Punitive']

X_pun_train, X_pun_val, y_pun_train, y_pun_val = sklearn.model_selection.train_test_split(X_pun, y_pun, test_size=0.2, random_state=42)

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = CountVectorizer(max_features = 20)
vectorizer.fit(X_lab_train)

dtm_lab_train = vectorizer.transform(X_lab_train)
dtm_lab_train.shape

dtm_lab_val = vectorizer.transform(X_lab_val)
dtm_lab_val.shape

(317, 20)

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = CountVectorizer(max_features = 20)
vectorizer.fit(X_pun_train)

dtm_pun_train = vectorizer.transform(X_pun_train)
dtm_pun_train.shape

dtm_pun_val = vectorizer.transform(X_pun_val)
dtm_pun_val.shape

(317, 20)

In [102]:
feature_names = vectorizer_pun.get_feature_names()
feature_names

['account',
 'content',
 'data',
 'including',
 'information',
 'may',
 'or',
 'party',
 'personal',
 'right',
 'service',
 'services',
 'site',
 'the',
 'third',
 'time',
 'use',
 'user',
 'we',
 'you']

# Build Models

### creates two models, one for punitive and one for labels

In [103]:
# Try diff models here

clf_lab = DecisionTreeClassifier(max_depth=2)
clf_lab.fit(dtm_lab_train, y_lab_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [104]:
# Try diff models here

clf_pun = DecisionTreeClassifier(max_depth=2)
clf_pun.fit(dtm_pun_train, y_pun_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

# Generate Output Vectors

In [147]:
# Label ouput
pred_val_labels = clf_lab.predict_proba(dtm_lab_val)

In [148]:
# punitive output
pred_val_pun = clf_pun.predict(dtm_pun_val)

In [149]:
# Turn max of label vector to 1, everything else to 0

b = np.zeros_like(pred_val_labels)
b[np.arange(len(pred_val_labels)), pred_val_labels.argmax(1)] = 1

In [164]:
# Concatenate Punitive and label vectors

Rating_per_sentence = np.column_stack((pred_val_pun, b))

In [166]:
Rating_per_sentence.shape

(317, 18)

In [168]:
X_lab_val.shape

(317,)

# Using Output Vectors Create Company Vector *DEPLOYMENT PART*

In [224]:
#choose company
company = 'twitch'

company_details = data_sentenced.loc[data_sentenced['Company'] == company]

In [225]:
# convert to model input vector

X_company = company_details['Sentence']

In [226]:
vectorizer = CountVectorizer(max_features = 20)
vectorizer.fit(X_company)
vectorized_company = vectorizer.transform(X_company)

In [227]:
# punitive output
pred_company_pun = clf_pun.predict(vectorized_company)

In [228]:
# Label ouput
pred_company_lab = clf_lab.predict_proba(vectorized_company)

pred_company_lab_one_hot = np.zeros_like(pred_company_lab)
pred_company_lab_one_hot[np.arange(len(pred_company_lab)), pred_company_lab.argmax(1)] = 1

In [230]:
mask = np.zeros([pred_company_lab_one_hot.shape[0], pred_company_lab_one_hot.shape[1]]) 

In [239]:
filtered_one_hot = np.where(pred_company_pun[:, None], pred_company_lab_one_hot, mask)

In [247]:
company_vector = filtered_one_hot.mean(axis=0)

In [248]:
#this will be compared with customer ratings
company_vector

array([0.        , 0.        , 0.        , 0.64285714, 0.        ,
       0.        , 0.        , 0.        , 0.28571429, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

# Calculate Cosine Similarity of customer array

In [262]:
darr_vector = np.array([0, 0, 0,5,0,10,0,0,0,0,0,0,0,0,0,0,0])

In [263]:
# normalize by diving array by 10 which is the highest number
darr_vector = darr_vector/10

In [264]:
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity 
    """
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    return dot_product / (norm_a * norm_b)

In [265]:
print(cos_sim(company_vector, darr_vector))

0.40866894826784983
